# ME 591 HW 3

Max Schrader

9/6/21

ME 591

In [1]:
import os
import math
import pint

In [2]:
def bordered(text):
    lines = text.splitlines()
    width = max(len(s) for s in lines)
    res = ['┌' + '─' * width + '┐']
    for s in lines:
        res.append('│' + (s + ' ' * width)[:width] + '│')
    res.append('└' + '─' * width + '┘')
    return '\n'.join(res)

## Problem 4-19

In [3]:
ureg = pint.UnitRegistry(autoconvert_offset_to_baseunit = True)

### Defining the Inputs

In [4]:
P_solar = 8 * ureg.month
P_heater = 4 * ureg.month

T_water_hot = 60 * ureg.degC
T_water_cold = 20 * ureg.degC
M_water = 6e3 * ureg.kilogram / ureg.month   # is this imperial tons or metric tons ? I am using metric tons bc everything else in the problem is metric
cost = 1.35 / (ureg.therm)
N_heater = 88 / 100
c_p = 4.18 * (ureg.kJ / (ureg.kilogram * ureg.delta_celsius)) 

### Solving for $Q$

In [5]:
Q = M_water * c_p * (T_water_hot - T_water_cold)
Q

1003200.0 <Unit('kilojoule / month')>

### Electricity Savings

In [6]:
e_savings = (Q / N_heater * P_solar).to('kJ')
e_savings

9120000.0 <Unit('kilojoule')>

### Cost Savings

In [7]:
print(bordered(f"Savings = ${round(float((e_savings * cost)), 2)}"))

┌────────────────┐
│Savings = $116.7│
└────────────────┘


## Problem 4-34 

In [8]:
ureg = pint.UnitRegistry(autoconvert_offset_to_baseunit = True)

In [9]:
CR = 7
N_ar = 0.88
T_collector = 145 * ureg.degC
T_air = 27 * ureg.degC

collectors = {
    'A': {
        'U': 2.5 * (ureg.watt / (ureg.meter ** 2 * ureg.delta_degC)),
        'G': 600 * (ureg.watt / (ureg.meter ** 2))
    },
    'B': {
        'U': 0.63 * (ureg.watt / (ureg.meter ** 2 * ureg.delta_degC)) 
    }
}


### Collector Efficiency of A

In [10]:
collectors['A']['N_c'] = N_ar - collectors['A']['U'] * (T_collector - T_air) / (CR * collectors['A']['G'])
collectors['A']['N_c']

0.8097619047619048 <Unit('dimensionless')>

### Finding Solar Irradation Rate of B

In [11]:
collectors['B']['N_c'] = collectors['A']['N_c']
collectors['B']['G'] = ((collectors['B']['N_c'] - N_ar) / (-1 * collectors['B']['U']) / (T_collector - T_air)) ** (-1) / CR
print(bordered(f"G_B = {round(collectors['B']['G'], 2)}"))

┌─────────────────────────────┐
│G_B = 151.2 watt / meter ** 2│
└─────────────────────────────┘


### Collector Efficiency of A if Solar Irradiation Increases

In [12]:
collectors['A']['G'] = 900 * (ureg.watt / (ureg.meter ** 2))
new_eff = N_ar - collectors['A']['U'] * (T_collector - T_air) / (CR * collectors['A']['G'])
print(bordered(f"Efficiency Change = {round((new_eff.magnitude - collectors['A']['N_c'].magnitude) * 100, 2)}%"))

┌─────────────────────────┐
│Efficiency Change = 2.34%│
└─────────────────────────┘


## Problem 4-45 

A solar-power-tower plant is considered for Houston, Texas. Heliostats with a total area of
400,000 ft 2 are to be used to reflect solar radiation into a receiver. When the solar irradiation is
250 Btu/h ⋅ ft 2 , steam is produced at 160 psia and 600°F at a rate of 15 lbm/s. This steam is expanded
in a turbine to 2 psia pressure. The isentropic efficiency of the turbine is 88 percent. (a) Determine
the power output, in kW and the thermal efficiency of the plant under these operating conditions.
(b) How much electricity can be produced per year if the average thermal efficiency is 12 percent and
the generator efficiency is 98 percent? Use the solar insolation value in Table 3-5 in Chap. 3. (c) The
estimated cost of this plant is $17,000/kW and the plant is expected to operate 4500 h a year at the
power output determined in part (a). If electricity generated is to be sold at a price of $0.11/kWh, how
long will it take for this plant to pay for itself?

In [13]:
ureg = pint.UnitRegistry(autoconvert_offset_to_baseunit = True)

In [14]:
A = (400_000 * ureg.ft ** 2).to(ureg.meters **2)
G = (250 * (ureg.BTU / (ureg.hour * ureg.ft ** 2))).to(ureg.watt / (ureg.meter ** 2))
P_steam_in = (160 * ureg.psi).to('MPa')
T_steam = (600 * ureg.degF).to('degC')
m_dot_steam = (15 * ureg.pound / ureg.second).to('kg/s')
P_steam_out = (2 * ureg.psi).to('MPa')
N_isentropic = .88

In [15]:
P_steam_in.to('kPa')

1103.1611669069382 <Unit('kilopascal')>

In [16]:
P_steam_out.to('kPa')

13.789514586336725 <Unit('kilopascal')>

In [17]:
T_steam

315.55555555555566 <Unit('degree_Celsius')>

### Part A

In [18]:
h1 = 3051.6 * ureg.kJ / ureg.kg  # (3051.6)
s1 = 7.1246 * ureg.kJ / (ureg.kg * ureg.degK) # (7.1246)

h2_s = 2590.7 * ureg.kJ / ureg.kg

In [19]:
h2 = h1 - N_isentropic * (h1 - h2_s)
h2

2646.008 <Unit('kilojoule / kilogram')>

In [20]:
W_out = m_dot_steam * (h1 - h2)
print(bordered(f"Power Output = {W_out.to('kW')}"))

┌──────────────────────────────────────────┐
│Power Output = 2759.6015479956013 kilowatt│
└──────────────────────────────────────────┘


In [21]:
N_th = W_out.to('watt') / (G * A)
print(bordered(f"Thermal Efficiency = {round(N_th.magnitude * 100, 2)}%"))

┌──────────────────────────┐
│Thermal Efficiency = 9.42%│
└──────────────────────────┘


### Part B

In [22]:
N_th_average = 0.12
G = 15.90 * ureg.megajoules / (ureg.m**2 * ureg.day)
N_gen = 0.98

In [23]:
W_out_B = N_th_average * A * G * (365 * ureg.day)
W_out_B.to('kWh')

7188837.2352 <Unit('kilowatt_hour')>

In [24]:
W_elec = N_gen * W_out_B
W_elec.to('kWh')
print(bordered(f"Electric Energy = {round(W_elec.to('kWh'), 2)} kWh"))

┌──────────────────────────────────────────────┐
│Electric Energy = 7045060.49 kilowatt_hour kWh│
└──────────────────────────────────────────────┘


### Part C

In [25]:
cost = W_out.to('kW') * (17_000 / (1 * ureg.kilowatt)).to('1/kW')
print(f"Cost = ${cost.magnitude}")

Cost = $46913226.315925226


In [26]:
yearly_revenue = (W_out * (4500 * ureg.hour)).to('kWh') * (0.11 / (ureg.kilowatt * ureg.hour)).to('1/kWh') 
print(f"Revenue = ${yearly_revenue.magnitude} per year")

Revenue = $1366002.7662578227 per year


In [27]:
print(bordered(f"Time to pay off = {round(cost.magnitude / yearly_revenue.magnitude, 2)} years"))

┌─────────────────────────────┐
│Time to pay off = 34.34 years│
└─────────────────────────────┘
